In [ ]:
#Question No.1
import nltk
from nltk.corpus import gutenberg
import re

In [248]:
def show_regex_matches(pattern, text):
  matches = re.findall(pattern, text)
  print(f"\nMatches for pattern {pattern}")
  print(' '.join(matches))
  print("Length: ", len(matches))

In [249]:
#Testing with a custom text
text = "Cat cat Hello coat cut cute Dog cart"
show_regex_matches(r'[a-z]+', text)
show_regex_matches(r'[A-Z][a-z]+', text)
show_regex_matches(r'c[aeiou]{1,2}t', text)


Matches for pattern [a-z]+
at cat ello coat cut cute og cart
Length:  8

Matches for pattern [A-Z][a-z]+
Cat Hello Dog
Length:  3

Matches for pattern c[aeiou]{1,2}t
cat coat cut cut
Length:  4


In [250]:
#Testing with gutenberg text
nltk.download('gutenberg')
text = gutenberg.raw('austen-emma.txt')
show_regex_matches(r'[a-z]+', text)
show_regex_matches(r'[A-Z][a-z]+', text)
show_regex_matches(r'c[aeiou]{1,2}t', text)



Matches for pattern [a-z]+
mma by ane usten mma oodhouse handsome clever and rich with a comfortable home and happy disposition seemed to unite some of the best blessings of existence and had lived nearly twenty one years in the world with very little to distress or vex her he was the youngest of the two daughters of a most affectionate indulgent father and had in consequence of her sister s marriage been mistress of his house from a very early period er mother had died too long ago for her to have more than an indistinct remembrance of her caresses and her place had been supplied by an excellent woman as governess who had fallen little short of a mother in affection ixteen years had iss aylor been in r oodhouse s family less as a governess than a friend very fond of both daughters but particularly of mma etween them it was more the intimacy of sisters ven before iss aylor had ceased to hold the nominal office of governess the mildness of her temper had hardly allowed her to impose an

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [251]:
#Question No.2
from urllib import request
from bs4 import BeautifulSoup

In [252]:
def get_text_from_url(url):
  html = request.urlopen(url).read().decode('utf8')
  return BeautifulSoup(html, 'html.parser').get_text()

In [253]:
url = "https://www.dsu.edu.pk/contact-us/"
text = get_text_from_url(url)
print(text.strip())

Contact Us - DHA Suffa University

























































TwitterLinkedInFacebookInstagramYoutubeRss        Online applications for Fall 2024 Semester for BE Mechanical Engineering, BE Electrical Engineering, BE Civil Engineering (Superhighway campus), BS Computer Science (Superhighway campus), BS Computer Science (Main campus) , BS Software Engineering (Main Campus), BS Engineering technology (Computer), BS Cyber Security (Main campus), BS Multimedia and Gaming (Main Campus), BS Data Science (Main Campus), BS (Artificial Intelligence) (Main Campus), BBA, BS (Business Analytics and Programming), BS (Accounting & Finance), BS (International Relations), BS (English), BS (Psychology), MPhil (Psychology), MBA, MS (MS/CS) and PhD (MS/ME/EE/CS) Programs are Open. Last date to apply and pay the fee is 6th September 2024 and on-campus test will be held on 10 September 2024(tentative) at MAIN CAMPUS DHA Phase 7ext. and Online test will be held on 11 September 20

In [254]:
#Question No.3
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [255]:
def get_word_tokens(text):
  return word_tokenize(text)

In [256]:
tokens = get_word_tokens(text)
print("Word Tokens: ", tokens)

Word Tokens:  ['Contact', 'Us', '-', 'DHA', 'Suffa', 'University', 'TwitterLinkedInFacebookInstagramYoutubeRss', 'Online', 'applications', 'for', 'Fall', '2024', 'Semester', 'for', 'BE', 'Mechanical', 'Engineering', ',', 'BE', 'Electrical', 'Engineering', ',', 'BE', 'Civil', 'Engineering', '(', 'Superhighway', 'campus', ')', ',', 'BS', 'Computer', 'Science', '(', 'Superhighway', 'campus', ')', ',', 'BS', 'Computer', 'Science', '(', 'Main', 'campus', ')', ',', 'BS', 'Software', 'Engineering', '(', 'Main', 'Campus', ')', ',', 'BS', 'Engineering', 'technology', '(', 'Computer', ')', ',', 'BS', 'Cyber', 'Security', '(', 'Main', 'campus', ')', ',', 'BS', 'Multimedia', 'and', 'Gaming', '(', 'Main', 'Campus', ')', ',', 'BS', 'Data', 'Science', '(', 'Main', 'Campus', ')', ',', 'BS', '(', 'Artificial', 'Intelligence', ')', '(', 'Main', 'Campus', ')', ',', 'BBA', ',', 'BS', '(', 'Business', 'Analytics', 'and', 'Programming', ')', ',', 'BS', '(', 'Accounting', '&', 'Finance', ')', ',', 'BS', '(',

In [257]:
#Extracting Phone Numbers from Text
pattern = r'(\(?\d{3,4}\)?[\s\-]?\d{7,8}[\s\-]?\d?\d?)'
print("Phone Numbers")
show_regex_matches(pattern, text)




Phone Numbers

Matches for pattern (\(?\d{3,4}\)?[\s\-]?\d{7,8}[\s\-]?\d?\d?)
021 35244851-52 0324-2444595 021-35244866  03218992494  021-35244875
 0345-2479903  021-35244855
 0307-8594065  021-35244862
 0323-2411025  021-35244874
 0306-3048354  021-35244851-2 0324-2444595  0345-2212938
 021-35244851-2 0324-2444595
 021-35244858
 0302-0241679
 021-35244857
 021-35244870  021-35244859
 0300-2595782  021-35244864
 021-35244853-4 021-35244853-4 021-35244853-4 021-35244853-4 0344-2412896
 021-35244851  021-35244853-4 021-35244853-4 021-35244865
 021-35244869
 021-35244853-4 0321-2162719
 021-35244871
 021-35244860
 0321-3680021
 021-35244855
 0340-4864835  021-35244861
 0346-3244745  021-35244872
 0304-2274288
 021-35244853-4 0322-2539007
 021-35244853-4 021-35244853-4 021-35244873
 021-35858604

Length:  51


In [258]:
#Extracting Emails from Text
pattern = r'[a-zA-Z0-9._%+-]+\@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
print("Emails")
show_regex_matches(pattern, text)

Emails

Matches for pattern [a-zA-Z0-9._%+-]+\@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}
ahmedsaeedminhas81@dsu.edu.pk vc@dsu.edu.pk mateen@dsu.edu.pk registrar@dsu.edu.pk dean.eas@dsu.edu.pk dean.mss@dsu.edu.pk saad.asif@dsu.edu.pk rabia.mumtaz@dsu.edu.pk a.registrar.acad@dsu.edu.pk hr@dsu.edu.pk farrukh.shahab@dsu.edu.pk sunil.kumar@dsu.edu.pk noor.ali@dsu.edu.pk mohammedrazamehdi@dsu.edu.pk sobia.iqbal@dsu.edu.pk razi@dsu.edu.pk rubina.israr@dsu.edu.pk sidra.ahmed@dsu.edu.pk m.zaman@dsu.edu.pk johar@dsu.edu.pk pdc@dsu.edu.pk saima@dsu.edu.pk subeika.rizvi@dsu.edu.pk subeika.rizvi@dsu.edu.pk director.ierc@dsu.edu.pk info.ceid@dsu.edu.pk saima@dsu.edu.pk aso@dsu.edu.pk j.khan@dsu.edu.pk ahmed.faseeh@dsu.edu.pk moin.atiq@dsu.edu.pk director.it@dsu.edu.pk hasan.raza@dsu.edu.pk a.mit.sys@dsu.edu.pk engr.riaz@dsu.edu.pk
Length:  35


In [259]:
#Question No.4
from nltk.stem import PorterStemmer, LancasterStemmer

In [260]:
text = "The runner was running and the cats were chasing mice."
tokens = get_word_tokens(text)
print("Word Tokens: ", tokens)

Word Tokens:  ['The', 'runner', 'was', 'running', 'and', 'the', 'cats', 'were', 'chasing', 'mice', '.']


In [261]:
porter = PorterStemmer()
porter_stemmed_tokens = [porter.stem(token) for token in tokens]
print("Porter Stemmed Tokens: ", porter_stemmed_tokens)

Porter Stemmed Tokens:  ['the', 'runner', 'wa', 'run', 'and', 'the', 'cat', 'were', 'chase', 'mice', '.']


In [262]:
lancaster = LancasterStemmer()
lancaster_stemmed_tokens = [lancaster.stem(token) for token in tokens]
print("Lancaster Stemmed Tokens: ", lancaster_stemmed_tokens)

Lancaster Stemmed Tokens:  ['the', 'run', 'was', 'run', 'and', 'the', 'cat', 'wer', 'chas', 'mic', '.']


In [263]:
#Question No.5
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


In [264]:
df = pd.read_csv('/content/Sentiment Dataset Urdu - Sentiment Dataset Urdu.csv', encoding='utf-8')
df.head()

,Plarity,Text
0,N,دو دن کی بارش میں سارا” بھرم“ بہہ گیا۔
1,N,مہذب دنیا میں حکومتیں اپنی پالیسیوں اور کارکردگی سے چلتی ہیں لیکن ہمارے ہاں حکومتوں کا سارا زور کارکردگی کے بجائے بھرم بازی پر ہوتا ہے۔
2,N,خوش کن وعدے ،لمبے چوڑے بیانات اور لیپا پوتی حکومتی عرصہ گزارنے کے لئے کافی ہوتی ہیں۔
3,N,اگر کوئی آفت سر پر آجائے تو پچھلی حکومتوں کو کوس کروقت گزار لیا جاتا ہے۔
4,N,یہی کچھ اس بار بھی ہوا جب دو دنوں کی بارش نے ملک کے اکثر علاقوں کو بلکہ بڑے بڑے شہروں کو تالاب میں تبدیل کر دیا۔


In [265]:
#text = df['Text'].to_string(index=False, header=False).strip().replace('\n', ' ')
text = df['Text']
print(text)

0                                                                                                                                                                                                                                                                                                                                                      دو دن کی بارش میں سارا” بھرم“ بہہ گیا۔
1                                                                                                                                                                                                                                                     مہذب دنیا میں حکومتیں اپنی پالیسیوں اور کارکردگی سے چلتی ہیں لیکن ہمارے ہاں حکومتوں کا سارا زور کارکردگی کے بجائے بھرم بازی پر ہوتا ہے۔
2                                                                                                                                                                                                                                           

In [266]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

In [267]:
tokenizer = Tokenizer(BPE())
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer.train_from_iterator(text, trainer)
vocab = tokenizer.get_vocab()
print("Vocab: ", vocab)
print("Vocab Size: ", len(vocab))

Vocab:  {'کفن': 3608, 'رکا': 2326, 'ؤنٹس': 3129, 'معام': 654, 'کوششیں': 2780, 'رائج': 3298, 'جیٹل': 910, 'گرمی': 1637, '6800': 5117, 'فائدہ': 1278, 'بض': 1496, 'حکمر': 707, 'صداقت': 4968, 'مرحلے': 4224, 'ریزنگ': 3829, 'پنکچر': 1747, 'ڈپار': 3584, 'ایسے': 388, 'ار': 103, 'لحاظ': 3041, 'جہانگی': 4212, 'سکتی': 699, 'تیوس': 3917, 'ساڑھ': 2678, 'منٹوں': 4948, 'آہ': 945, 'اپن': 2577, 'بننے': 1667, 'مرہ': 2415, 'پہچان': 1674, 'دناک': 3251, 'شرمندہ': 4382, 'محترمہ': 5577, 'دیکھی': 3877, 'مقت': 3468, 'خلے': 1310, 'وری': 268, 'مختار': 4452, 'ہوتی': 296, 'کو': 99, 'دوں': 449, 'آرڈر': 4674, 'عمارت': 5164, 'داخلے': 1415, 'چیف': 850, 'شتہ': 456, 'دنوں': 4262, 'نگر': 2425, 'سنگ': 3327, 'لنج': 1541, 'جیمر': 4088, 'گم': 2473, 'بٹنوں': 5289, 'فارم': 4813, 'بدنام': 3019, 'انگریزوں': 4938, 'لمبے': 2818, 'چھوڑتے': 4734, 'گوں': 1931, 'الیکٹر': 2961, 'طولیٰ': 5158, 'گن': 1198, 'آواز': 1031, 'بیگ': 1643, 'کاکہنا': 3708, 'منافع': 2220, 'یقینا': 1281, '“': 78, 'کورٹ': 2908, 'دگرگوں': 5326, 'لاشع': 4135, 'ضروری'

In [268]:
output_tokens = tokenizer.encode("پچھلی بارشوں کے باعث شہر کی سڑکیں تالاب میں تبدیل ہو گئیں اور حکومتیں کچھ نہ کر سکیں۔ شہریوں کو اپنی املاک اور جانوں کا نقصان اٹھانا پڑا۔ زندگی مفلوج ہو گئی اور روزمرہ کی مشکلات میں اضافہ ہوا۔")
print("Output Tokens: ", output_tokens.tokens)
print("Output Tokens Size: ", len(output_tokens))
bpe_tokens = output_tokens.tokens

Output Tokens:  ['پچھلی', 'بارشوں', 'کے', 'باعث', 'شہر', 'کی', 'سڑک', 'یں', 'تالاب', 'میں', 'تبدیل', 'ہو', 'گئیں', 'اور', 'حکومتیں', 'کچھ', 'نہ', 'کر', 'سکیں', '۔', 'شہریوں', 'کو', 'اپنی', 'املاک', 'اور', 'جان', 'وں', 'کا', 'نقصان', 'اٹھانا', 'پڑا', '۔', 'زندگی', 'مفلوج', 'ہو', 'گئی', 'اور', 'روزمرہ', 'کی', 'مشکلات', 'میں', 'اضافہ', 'ہوا', '۔']
Output Tokens Size:  44


In [269]:
dictionary = set()
for key, value in vocab.items():
  dictionary.add(key)

In [270]:
def max_match_segment(text, dictionary):
    segmented = []
    i = 0
    while i < len(text):
        for j in range(len(text), i, -1):
            word = text[i:j]
            for dic in dictionary:
                if dic == word:
                    segmented.append(word)
                    i = j
                    break
        else:
            segmented.append(text[i])
            i += 1
    return segmented


In [271]:
text = "پچھلی بارشوں کے باعث شہر کی سڑکیں تالاب میں تبدیل ہو گئیں اور حکومتیں کچھ نہ کر سکیں۔ شہریوں کو اپنی املاک اور جانوں کا نقصان اٹھانا پڑا۔ زندگی مفلوج ہو گئی اور روزمرہ کی مشکلات میں اضافہ ہوا۔"
segmented_words = max_match_segment(text, dictionary)
print("Output Tokens:", segmented_words)
print("Output Tokens Size: ", len(segmented_words))
mm_tokens = segmented_words

Output Tokens: ['پچھلی', ' ', 'بارشوں', ' ', 'کے', ' ', 'باعث', ' ', 'شہر', ' ', 'کی', ' ', 'سڑک', 'ی', 'ں', ' ', 'تالاب', ' ', 'میں', ' ', 'تبدیل', ' ', 'ہو', ' ', 'گئیں', ' ', 'اور', ' ', 'حکومتیں', ' ', 'کچھ', ' ', 'نہ', ' ', 'کر', ' ', 'سکیں', '۔', ' ', 'شہریوں', ' ', 'کو', ' ', 'اپنی', ' ', 'املاک', ' ', 'اور', ' ', 'جان', 'و', 'ں', ' ', 'کا', ' ', 'نقصان', ' ', 'اٹھانا', ' ', 'پڑا', '۔', ' ', 'زندگی', ' ', 'مفلوج', ' ', 'ہو', ' ', 'گئی', ' ', 'اور', ' ', 'روزمرہ', ' ', 'کی', ' ', 'مشکلات', ' ', 'میں', ' ', 'اضافہ', ' ', 'ہوا', '۔']
Output Tokens Size:  84


In [272]:
import numpy as np
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.tokenize import word_tokenize
from collections import defaultdict
import pandas as pd

def train_lm(train_tokens, n):
    train_data, padded_vocab = padded_everygram_pipeline(n, [train_tokens])
    print("Training Padded Vocabulary:", list(padded_vocab))
    for data in train_data:
        print("Training Data: ", list(data))
    model = MLE(n)
    model.fit(train_data, padded_vocab)
    return model

def compute_perplexity(model, test_tokens, n):
    test_data, _ = padded_everygram_pipeline(n, [test_tokens])
    total_log_prob = 0
    num_tokens = 0

    for data in test_data:
        num_tokens += len(list(data))
        for ngram in list(data):
            prob = model.score(ngram[-1], *ngram[:-1])  # For n-grams
            total_log_prob += np.log(prob)

    # Calculate perplexity
    entropy = -total_log_prob / num_tokens
    perplexity = np.exp2(entropy)

    return perplexity

train_tokens = list(dictionary)
print("Train Tokens: ", train_tokens)

test_text = "دو دن کی بارش میں سارا” بھرم“ بہہ گیا۔"
test_tokens = get_word_tokens(test_text)
print("Test Tokens: ", test_tokens)

# Train unigram model
unigram_model = train_lm(train_tokens, 1)
unigram_perplexity = compute_perplexity(unigram_model, test_tokens, 1)
print("Unigram Model Perplexity:", unigram_perplexity)

# Train bigram model
bigram_model = train_lm(train_tokens, 2)
bigram_perplexity = compute_perplexity(bigram_model, test_tokens, 2)
print("Bigram Model Perplexity:", bigram_perplexity)

# Train trigram model
trigram_model = train_lm(train_tokens, 3)
trigram_perplexity = compute_perplexity(trigram_model, test_tokens, 3)
print("Trigram Model Perplexity:", trigram_perplexity)


Train Tokens:  ['جانتی', 'انسانوں', 'ممکنہ', 'دھری', 'الیکٹر', 'ائیں', 'یوسف', 'درحقیقت', 'مات', 'اج', 'ہٹالئے', 'برباد', 'جلو', 'معیشیت', 'غرق', 'جواب', 'چوٹی', 'ساختہ', 'ہزار', 'نمٹنے', 'صوبے', 'مدافعت', 'سہن', 'میہ', 'دیکھ', 'ڈسپلے', 'غت', 'بگول', 'تما', 'پاک', 'دیکھو', 'زخم', 'دیکھے', 'دورانیہ', 'ٹریک', 'قاعد', 'حویلی', 'روف', 'ریں', 'بیماریوں', 'جاوید', 'فاقوں', 'دھی', 'جمال', 'سیٹ', 'بارشوں', 'لائبری', 'دئیے', 'الزام', 'اوائل', 'فخر', 'طر', 'جذبے', 'اٹھلا', 'سلجھا', 'ٹی', 'میٹنگز', 'ھی', 'رک', 'رمرد', 'ظالم', 'زبانی', 'غیرقان', 'فیزیا', 'بلدیاتی', 'اری', 'ستان', 'انڈی', 'طبع', 'لائے', 'صنفی', 'منڈر', 'طور', 'بلاگ', 'کوبی', 'زمین', 'برا', 'افسرچوہدری', 'لال', 'ندگی', 'کج', 'ایٹمی', 'پرانے', 'ستم', 'ازت', 'اکاؤنٹس', 'موٹر', 'سڑکوں', 'ضی', 'خانوں', 'کھونٹا', 'منصب', 'گوں', '3ہزار', 'من', 'قلیل', 'برسوں', 'نمائندے', 'میدانوں', 'پالیسیاں', 'علاج', 'کتنی', 'ربا', 'لگدے', 'سو', 'خوشامد', 'انتقام', 'ناموافق', 'ٹو', 'چاہئیں', 'انیاں', 'چرب', 'جواپنی', 'مائی', 'تجربات', 'دگرگوں', 'جانوروں'